In [ ]:
import pandas as pd
import tabula
import PyPDF2
import regex as re
from urllib3 import request
from urllib.request import Request, urlopen
import requests
from requests.auth import HTTPBasicAuth
import io
from io import StringIO
import os
import xlrd
import fitz # install using: pip install PyMuPDF

In [ ]:
# Magic numbers
regular_res_len = 3

In [ ]:
def get_page_and_header(target_fst, target_lst, file):
    doc = fitz.open(file)
    res = {}
    for page in doc:
        text = page.get_text()
        text = text.replace("\n", "")
        if page.number == 0:
            res['fund_name'] = text.strip()
        if re.search(target_fst, text):
            res['fst_page'] = int(str(page).split(" ")[1])
        elif re.search(target_lst, text):
            res['lst_page'] = int(str(page).split(" ")[1])
    return res

In [ ]:
# find all the file names in the target folder
def file_names(dir_path):
    files = []
    for path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, path)):
            files.append(path)
    return files #e.g. '20201028-nxt-im.pdf'

In [ ]:
dir_path = r"C:\Users\yongyue.song\Downloads\memo_pdf"
target_fst = "Appendix\s..\sFund\sTerms" #e.g. Appendix X: Fund Terms
target_lst = "Legal\sDue\sDiligence"
files = file_names(dir_path)
file_paths = [os.path.join(dir_path, file_name) for file_name in files] #e.g. 'C:\\Users\\yongyue.song\\Downloads\\memo_test\\20201028-nxt-im.pdf'

In [ ]:
# dir_path = r"C:\Users\yongyue.song\Downloads\memo_chinese"
# target_fst = "附录.+主要投资条款" #e.g. Appendix X: Fund Terms
# target_lst = "咨询顾问委员会"
# files = file_names(dir_path)
# file_paths = [os.path.join(dir_path, file_name) for file_name in files] #e.g. 'C:\\Users\\yongyue.song\\Downloads\\memo_test\\20201028-nxt-im.pdf'

In [ ]:
for i in range(len(file_paths)): 
    curr_file = file_paths[i]
    res = get_page_and_header(target_fst, target_lst, curr_file)
    
    # save pdfs in a folder
    folder_name = 'legal_terms'
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    
    # for those pdfs that are formatted differently 
    if len(res) != regular_res_len:
        fund_legal_terms = pd.DataFrame([curr_file])
        fund_legal_terms.to_excel(os.path.join(folder_name, f"{i}_missing.xlsx"), index=False)
    else: 
        fst_page_num = res['fst_page'] 
        lst_page_num = res['lst_page']
        fund_name = res['fund_name']
        tables = tabula.read_pdf(curr_file, 
                                 pages="{fst}-{lst}".format(fst = fst_page_num, lst = lst_page_num+1),
                                 pandas_options={'header': None})
        df = pd.concat(tables)

        # add fund name to the first row of the table
        n_df = pd.DataFrame([['Fund:', fund_name]])
        fund_legal_terms = pd.concat([n_df, df], ignore_index=True)
        fund_legal_terms.to_excel(os.path.join(folder_name, f"{files[i]}.xlsx"), index=False)
        